In [492]:
import re
from itertools import groupby
from datetime import datetime

In [493]:
def returnValue(val):
    return val.group(1) if val else None

In [494]:
def isPredictMsg(msg):
    patterns = [
        r"Symbol: (.+)",
        r"Position: (.+)",
        r"Leverage: (.+)",
        r"Market: (.+)",
        r"StopLoss: (.+)",
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg) for pattern in patterns)

In [495]:
def findPosition(msg):
    pos = None
    if "short" in msg.lower():
        pos = "SHORT"
    elif "long" in msg.lower():
        pos = "LONG"

    return pos

In [496]:
def findMarket(msg):
    market = None
    if "Futures Call".lower() in msg.lower():
        market = "FUTURES"
    elif "Spot".lower() in msg.lower():
        market = "SPOT"

    return market

In [497]:
msg = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "📣 Futures Call\n\n  💹SHORT\n\n📌 #LINKUSDT\n\n💰 Entry: 14.56-15.8\n\n🏹 TP:14.35-14.1-13.8-13.5-13.35-13\n12.5-12-11.6-11.2-10.7-10-8\n\n🧨 Stop : کلوز 16.54\nU\n⚖️ Leverage: 2x-3x-4X\n\n✅ Capital %: 3-5\n\nسیگنال قبل سود خوبی داد بنظر برگشتها پولبکه مجدد میشه وارد شد\n\n1402.09.28\n〰️〰️〰️〰️〰️ ️",  
    "edit_date": None,
    "media": None,
}

string = msg["message"]
# print(string)
symbol_match = re.search(r"📌 #(.+)", string)
position_match = findPosition(string)
leverage_match = re.search(r"Leverage: (.+)", string)
market_match = findMarket(string)
stopLoss_match = re.search(r"Stop : (.+)", string)

# Extracting values from entry targets
entry_targets_match = re.search(r"Entry:(.+?)\n\n", string, re.DOTALL)
entry_values = (
    re.findall(r"\d+(?:\.\d+)?", entry_targets_match.group(1))
    if entry_targets_match
    else None
)

take_profit_targets_match = re.search(r"TP:(.+?)\n\n", string, re.DOTALL)
profit_values = (
    re.findall(r"\d+(?:\.\d+)?", take_profit_targets_match.group(1))
    if take_profit_targets_match
    else None
)


# Creating a dictionary
data = {
    "Symbol": returnValue(symbol_match),
    "Position": position_match,
    "Market": market_match,
    "Leverage": returnValue(leverage_match),
    "StopLoss": returnValue(stopLoss_match),
    "Entry Targets": [
        {"index": i, "value": value, "active": False, "Period": None, "date": None}
        for i, value in enumerate(entry_values)
    ],
    "Take-Profit Targets": [
        {"index": i, "value": value, "active": False, "Period": None, "date": None}
        for i, value in enumerate(profit_values)
    ]
    if profit_values
    else None,
}
data
# print(data)

{'Symbol': 'LINKUSDT',
 'Position': 'SHORT',
 'Market': 'FUTURES',
 'Leverage': '2x-3x-4X',
 'StopLoss': 'کلوز 16.54',
 'Entry Targets': [{'index': 0,
   'value': '14.56',
   'active': False,
   'Period': None,
   'date': None},
  {'index': 1,
   'value': '15.8',
   'active': False,
   'Period': None,
   'date': None}],
 'Take-Profit Targets': [{'index': 0,
   'value': '14.35',
   'active': False,
   'Period': None,
   'date': None},
  {'index': 1, 'value': '14.1', 'active': False, 'Period': None, 'date': None},
  {'index': 2, 'value': '13.8', 'active': False, 'Period': None, 'date': None},
  {'index': 3, 'value': '13.5', 'active': False, 'Period': None, 'date': None},
  {'index': 4,
   'value': '13.35',
   'active': False,
   'Period': None,
   'date': None},
  {'index': 5, 'value': '13', 'active': False, 'Period': None, 'date': None},
  {'index': 6, 'value': '12.5', 'active': False, 'Period': None, 'date': None},
  {'index': 7, 'value': '12', 'active': False, 'Period': None, 'date': 

In [499]:
def isTakeProfit(msg, symbol, index):
    patterns = [
        rf"Full targets achieved Tp{index}(.+)",
        rf"{symbol}",
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg) for pattern in patterns)

In [500]:
def subtractTime(date1, date2):
    # Convert the datetime strings to datetime objects
    datetime1 = datetime.fromisoformat(date1)
    datetime2 = datetime.fromisoformat(date2)

    # Calculate the difference between the datetime objects
    time_difference = datetime2 - datetime1

    # Extract the hours and minutes from the time difference
    hours = time_difference.seconds // 3600
    minutes = (time_difference.seconds // 60) % 60

    # Format the result
    result = f"{hours} Hours {minutes} Minutes"
    return result

In [501]:
predict_messages = [
    {
        "id": 3083,
        "date": "2023-12-19T12:21:33+00:00",
        "reply_to_msg_id": None,
        "message": "📣 Futures Call\n\n  💹SHORT\n\n📌 #LINKUSDT\n\n💰 Entry: 14.56-15.8\n\n🏹 TP:14.35-14.1-13.8-13.5-13.35-13\n12.5-12-11.6-11.2-10.7-10-8\n\n🧨 Stop : کلوز 16.54\nU\n⚖️ Leverage: 2x-3x-4X\n\n✅ Capital %: 3-5\n\nسیگنال قبل سود خوبی داد بنظر برگشتها پولبکه مجدد میشه وارد شد\n\n1402.09.〰️", 
        "edit_date": None,
        "is_predict_msg": True,
        "predict": {
            "Symbol": "#LINKUSDT",
            "Position": "SHORT",
            "Market": "FUTURES",
            "Leverage": "2x-3x-4X",
            "StopLoss": "کلوز 16.54",
            "Entry Targets": [
                {
                    "index": 0,
                    "value": "14.56",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
                {
                    "index": 1,
                    "value": "15.8",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
            ],
            "Take-Profit Targets": [
                {
                    "index": 0,
                    "value": "14.35",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
                {
                    "index": 1,
                    "value": "14.1",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
                {
                    "index": 2,
                    "value": "13.8",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
                {
                    "index": 3,
                    "value": "13.5",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
            ],
            "status": "pending",
        },
        "media": None,
    }
]

grouped_data = {
    "3083": [
        {
            "id": 3090,
            "date": "2023-12-19T18:27:16+00:00",
            "reply_to_msg_id": 3083,
            "message": "#LINKUSDT Full targets achieved Tp2✅18%\n\n1402.09.28\n〰️〰️〰️〰️〰️〰️",
            "edit_date": None,
            "is_predict_msg": False,
            "predict": None,
            "media": None,
        },
        {
            "id": 3085,
            "date": "2023-12-19T15:14:46+00:00",
            "reply_to_msg_id": 3083,
            "message": "#LINKUSDT Full targets achieved Tp1✅13%\n\n1402.09.28\n〰️〰️〰️〰️〰️〰️",
            "edit_date": None,
            "is_predict_msg": False,
            "predict": None,
            "media": None,
        },
    ],
}

#  ali beyranvand does not sent entry msg
for msg in predict_messages:
    entries = msg["predict"]["Entry Targets"]
    take_profits = msg["predict"]["Take-Profit Targets"]
    symbol = msg["predict"]["Symbol"]

    all_predicts = grouped_data[f'{msg["id"]}']
    isPredictActive = False
    for j, item in enumerate(take_profits):
        for i, value in enumerate(all_predicts):
            if isTakeProfit(value["message"], symbol, j + 1):
                item["active"] = True
                item["date"] = value["date"]
                item["Period"] = subtractTime(msg["date"], value["date"])
                del all_predicts[i]
                isPredictActive = True
                break

    if isPredictActive:
        for item in entries:
            item["active"] = True
            item["date"] = None
            item["Period"] = None


predict_messages[0]

{'id': 3083,
 'date': '2023-12-19T12:21:33+00:00',
 'reply_to_msg_id': None,
 'message': '📣 Futures Call\n\n\xa0 💹SHORT\n\n📌 #LINKUSDT\n\n💰 Entry: 14.56-15.8\n\n🏹 TP:14.35-14.1-13.8-13.5-13.35-13\n12.5-12-11.6-11.2-10.7-10-8\n\n🧨 Stop : کلوز 16.54\nU\n⚖️ Leverage: 2x-3x-4X\n\n✅ Capital %: 3-5\n\nسیگنال قبل سود خوبی داد بنظر برگشتها پولبکه مجدد میشه وارد شد\n\n1402.09.〰️',
 'edit_date': None,
 'is_predict_msg': True,
 'predict': {'Symbol': '#LINKUSDT',
  'Position': 'SHORT',
  'Market': 'FUTURES',
  'Leverage': '2x-3x-4X',
  'StopLoss': 'کلوز 16.54',
  'Entry Targets': [{'index': 0,
    'value': '14.56',
    'active': True,
    'Period': None,
    'date': None},
   {'index': 1,
    'value': '15.8',
    'active': True,
    'Period': None,
    'date': None}],
  'Take-Profit Targets': [{'index': 0,
    'value': '14.35',
    'active': True,
    'Period': '2 Hours 53 Minutes',
    'date': '2023-12-19T15:14:46+00:00'},
   {'index': 1,
    'value': '14.1',
    'active': True,
    'Period': '6 

In [502]:
{
    "id": 3083,
    "date": "2023-12-19T12:21:33+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #GMX/USDT\nMarket: FUTURES\nPosition: SHORT\nLeverage: Isolated 12x\n\nEntry Targets: \n1) 51.07\n2) 51.41\n\nTake-Profit Targets: \n1) 50.80\n2) 50.51\n3) 50.25\n4) 49.80\n\nStopLoss: 51.70\nB.z",
    "edit_date": None,
    "is_predict_msg": True,
    "predict": {
        "Symbol": "#LINKUSDT",
        "Position": "SHORT",
        "Market": "FUTURES",
        "Leverage": "2x-3x-4X",
        "StopLoss": "کلوز 16.54",
        "Entry Targets": [
            {
                "index": 0,
                "value": "14.56",
                "active": False,
                "Period": None,
                "date": None,
            },
            {
                "index": 1,
                "value": "15.8",
                "active": False,
                "Period": None,
                "date": None,
            },
        ],
        "Take-Profit Targets": [
            {
                "index": 0,
                "value": "14.35",
                "active": False,
                "Period": None,
                "date": None,
            },
            {
                "index": 1,
                "value": "14.1",
                "active": False,
                "Period": None,
                "date": None,
            },
            {
                "index": 2,
                "value": "13.8",
                "active": False,
                "Period": None,
                "date": None,
            },
            {
                "index": 3,
                "value": "13.5",
                "active": False,
                "Period": None,
                "date": None,
            },
            {
                "index": 4,
                "value": "13.35",
                "active": False,
                "Period": None,
                "date": None,
            },
            {"index": 5, "value": "13", "active": False, "Period": None, "date": None},
            {
                "index": 6,
                "value": "12.5",
                "active": False,
                "Period": None,
                "date": None,
            },
            {"index": 7, "value": "12", "active": False, "Period": None, "date": None},
            {
                "index": 8,
                "value": "11.6",
                "active": False,
                "Period": None,
                "date": None,
            },
            {
                "index": 9,
                "value": "11.2",
                "active": False,
                "Period": None,
                "date": None,
            },
            {
                "index": 10,
                "value": "10.7",
                "active": False,
                "Period": None,
                "date": None,
            },
            {"index": 11, "value": "10", "active": False, "Period": None, "date": None},
            {"index": 12, "value": "8", "active": False, "Period": None, "date": None},
        ],
        "status": "pending",
    },
    "media": None,
}

{'id': 3083,
 'date': '2023-12-19T12:21:33+00:00',
 'reply_to_msg_id': None,
 'message': 'Symbol: #GMX/USDT\nMarket: FUTURES\nPosition: SHORT\nLeverage: Isolated 12x\n\nEntry Targets: \n1) 51.07\n2) 51.41\n\nTake-Profit Targets: \n1) 50.80\n2) 50.51\n3) 50.25\n4) 49.80\n\nStopLoss: 51.70\nB.z',
 'edit_date': None,
 'is_predict_msg': True,
 'predict': {'Symbol': '#LINKUSDT',
  'Position': 'SHORT',
  'Market': 'FUTURES',
  'Leverage': '2x-3x-4X',
  'StopLoss': 'کلوز 16.54',
  'Entry Targets': [{'index': 0,
    'value': '14.56',
    'active': False,
    'Period': None,
    'date': None},
   {'index': 1,
    'value': '15.8',
    'active': False,
    'Period': None,
    'date': None}],
  'Take-Profit Targets': [{'index': 0,
    'value': '14.35',
    'active': False,
    'Period': None,
    'date': None},
   {'index': 1,
    'value': '14.1',
    'active': False,
    'Period': None,
    'date': None},
   {'index': 2,
    'value': '13.8',
    'active': False,
    'Period': None,
    'date': No